### Scraping fighters data

In [2]:
import requests,bs4

In [12]:
BASE_URL = "http://ufcstats.com/statistics/fighters"

In [67]:
import string

# The page is organized by fighters names so I decided to
# Loop through different pages using an alphabets list
alphabets = list(string.ascii_lowercase)
for letter in alphabets:
    fighters_DOM_per_letter = requests.get(f"{BASE_URL}?char={letter}")
    soup = bs4.BeautifulSoup(fighters_DOM_per_letter.text, 'html.parser')
    pagination = soup.find('ul', class_='b-statistics__paginate')
    n_pages = len(pagination.find_all('li'))

    # Initialize the fighters dict
    if letter == 'a':
        headers = [th.get_text(strip=True) for th in soup.select("table thead th")]
        fighters_data = {header: [] for header in headers}
    
    # Not adding 1 to the number of list items because
    # there is a link for "all"
    for i in range(1, n_pages):
        current_page = requests.get(f"{BASE_URL}?char={letter}&page={i}")
        soup_1 = bs4.BeautifulSoup(current_page.text, 'html.parser')

        # Scraping fighters tabular data 
        for row in soup_1.select("table tbody tr"):
            cells = row.find_all('td',class_='b-statistics__table-col')
            
            if len(cells) == 0 :
                continue
            
            while len(cells) < len(headers):
                cells.append(None)
            
            for header,cell in zip(headers,cells):
                if cell:
                    fighters_data[header].append(cell.get_text(strip=True))
                else :
                    fighters_data[header].append(None)

In [68]:
import pandas as pd

df = pd.DataFrame(fighters_data)

In [71]:
df.tail()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
3562,Zhang,Weili,Magnum,"5' 4""",115 lbs.,"63.0""",Switch,26,3,0,
3563,Zhang,Mingyang,Mountain Tiger,"6' 2""",205 lbs.,"75.0""",Orthodox,19,7,0,
3564,Zhang,Qinghe,,"5' 7""",135 lbs.,"67.0""",Southpaw,16,8,1,
3565,Zhang,Zhenyang,,--,155 lbs.,--,,7,2,0,
3566,Daermisi,Zhawupasi,,"5' 8""",135 lbs.,"68.0""",Switch,9,2,0,


### Scraping Events Data

In [76]:
BASE_URL = "http://ufcstats.com/statistics/events/completed?page=all"

In [ ]:
events_page = requests.get("http://ufcstats.com/statistics/events/completed?page=all")

In [ ]:
fights_soup = bs4.BeautifulSoup(events_page.text,'html.parser')

# Get table headers
events_table_headers = [th.get_text(strip=True) for th in fights_soup.select('table thead th')]
# The first header is Name/Date so it is better to split it into separate name & date
events_table_headers = events_table_headers[0].split('/')+events_table_headers[1:]
events_dict = {header:{} for header in events_table_headers}
rows = fights_soup.select('table tbody tr.b-statistics__table-row')
# The first element is an empty row  
rows.pop(0)
rows[0].children

# for row in rows :
#     event_name = row.children